데이터: https://www.data.go.kr/data/15075057/openapi.do

# 데이터 저장

** 변수 의미 **

1. 업체명: entpName
2. 제품명: itemName
3. 효능: efcyQesitm
4. 사용법: useMethodQesitm
5. 주의사항 경고: atpnWarnQesitm
6. 주의사항: atpnQesitm
7. 상호작용: intrcQesitm
8. 부작용: seQesitm

In [ ]:
import requests
import urllib.parse
import csv

In [ ]:
api_key = urllib.parse.quote('''발급받은 디코딩 키''')

with open("./medicine_info.csv", "a", newline="", encoding="utf-8-sig") as f:
  writer = csv.writer(f)
  writer.writerow(["업체명", "제품명", "효능", "사용법", "주의사항 경고", "주의사항", "상호작용", "부작용"])

  for pageNo in range(1,50):
    url = f"http://apis.data.go.kr/1471000/DrbEasyDrugInfoService/getDrbEasyDrugList?serviceKey={api_key}&numOfRows=100&pageNo={pageNo}&type=json"

    response = requests.get(url)
    data = response.json()

    items = data.get("body", {}).get("items", [])

    for item in items:
        writer.writerow([
            item.get("entpName", ""),
            item.get("itemName", ""),
            item.get("efcyQesitm", ""),
            item.get("useMethodQesitm", ""),
            item.get("atpnWarnQesitm", ""),
            item.get("atpnQesitm", ""),
            item.get("intrcQesitm", ""),
            item.get("seQesitm", "")
        ])

# 데이터 임베딩하기

In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F
import psycopg2
import numpy as np

## 데이터 준비

In [ ]:
df = pd.read_csv("medicine_info.csv")

In [ ]:
df_filtered = df[df['효능'].notna() & (df['효능'] != "")]

In [ ]:
df_filtered = df.fillna('')

In [ ]:
df_filtered = df_filtered.replace({"\n": " ", "\xa0": " "}, regex=True)

In [ ]:
texts = (df_filtered['효능'] + " " +
         df_filtered['사용법'] + " " +
         df_filtered['주의사항 경고'] + " " +
         df_filtered['주의사항'] + " " +
         df_filtered['상호작용'])

## 임베딩

In [ ]:
MODEL_NAME = "dragonkue/snowflake-arctic-embed-l-v2.0-ko"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.eval()

In [ ]:
def get_embedding(text):
  inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)

  inputs = {k: v.to(device) for k, v in inputs.items()}

  with torch.no_grad():
      outputs = model(**inputs)
      embedding = outputs.last_hidden_state.mean(dim=1)
      embedding = F.normalize(embedding, p=2, dim=1)

  return embedding.cpu().numpy().flatten()

In [ ]:
embeddings = []
for text in texts:
  emb = get_embedding(text)
  embeddings.append(emb)

In [ ]:
embeddings = np.vstack(embeddings)
df_filtered['embedding'] = embeddings.tolist()

In [ ]:
print("Embeddings shape:", embeddings.shape)

Embeddings shape: (4840, 1024)


# 데이터 SQL문으로 만들기

In [ ]:
id = 1

In [ ]:
f1 = open('DBEM_Embedding_SQL.txt', 'w')
f2 = open('DBEM_Drug_SQL.txt', 'w')

In [ ]:
for row in zip(df_filtered['업체명'], df_filtered['제품명'], df_filtered['효능'], df_filtered['사용법'], df_filtered['주의사항 경고'], df_filtered['주의사항'], df_filtered['상호작용'], df_filtered['부작용'], df_filtered['embedding']):
  # embedding
  s = f'INSERT INTO drug_embedding VALUES ({id}, \'{row[0]}\', \'{row[1]}\', \'{row[8]}\'\n);'
  f1.write(s)

  # data
  s = f'INSERT INTO drug VALUES ({id}, \'{row[0]}\', \'{row[1]}\', \'{row[2]}\', '

  if row[3] != '':
    s += f'\'{row[3]}\', '
  else:
    s += '\'\', '

  if row[4] != '':
    s += f'\'{row[4]}\', '
  else:
    s += '\'\', '

  if row[5] != '':
    s += f'\'{row[5]}\', '
  else:
    s += '\'\', '

  if row[6] != '':
    s += f'\'{row[6]}\', '
  else:
    s += '\'\', '

  if row[7] != '':
    s += f'\'{row[7]}\');\n'
  else:
    s += '\'\');\n'

  f2.write(s)

  id += 1

In [ ]:
f1.close()
f2.close()